In [1]:
import sys
import pandas as pd
import numpy as np
import pickle
import pytz
import joblib
import yfinance as yf
import seaborn as sns
sns.set(style='darkgrid')
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
from dateutil.relativedelta import relativedelta

# for reproducibility of our results
np.random.seed(1234)

from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
import prophet

sys.path.append('../src') 
from utils import stock_trading_strategy, stock_trading_strategy_supp_resist, plot_trading_strategy, download_stock_df, user_function, plot_candlestick

ModuleNotFoundError: No module named 'tensorflow'

In [3]:
!pip install tensorflow


ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)
ERROR: No matching distribution found for tensorflow
